In [1]:
import numpy as np
import os
import six.moves.urllib as urllib
import sys
import tarfile
import tensorflow as tf
import zipfile

from collections import defaultdict
from io import StringIO
from matplotlib import pyplot as plt
from PIL import Image

c:\users\neark\realtimeshipdetector\env\lib\site-packages\tensorflow\python\framework\dtypes.py:523: FutureWarning: Passing (type, 1) or '1type' as a synonym of type is deprecated; in a future version of numpy, it will be understood as (type, (1,)) / '(1,)type'.
  _np_qint8 = np.dtype([("qint8", np.int8, 1)])
c:\users\neark\realtimeshipdetector\env\lib\site-packages\tensorflow\python\framework\dtypes.py:524: FutureWarning: Passing (type, 1) or '1type' as a synonym of type is deprecated; in a future version of numpy, it will be understood as (type, (1,)) / '(1,)type'.
  _np_quint8 = np.dtype([("quint8", np.uint8, 1)])
c:\users\neark\realtimeshipdetector\env\lib\site-packages\tensorflow\python\framework\dtypes.py:525: FutureWarning: Passing (type, 1) or '1type' as a synonym of type is deprecated; in a future version of numpy, it will be understood as (type, (1,)) / '(1,)type'.
  _np_qint16 = np.dtype([("qint16", np.int16, 1)])
c:\users\neark\realtimeshipdetector\env\lib\site-packages\ten

In [2]:
%matplotlib inline
sys.path.append("..")

In [3]:
from object_detection.utils import label_map_util
from object_detection.utils import visualization_utils as vis_util

In [4]:
MODEL_NAME = 'object_detection/saved_models/ssd_mobilenet_ship_v1'
PATH_TO_CKPT = MODEL_NAME + '/frozen_inference_graph.pb'
PATH_TO_LABELS = os.path.join('object_detection/data', 'ship_label_map.pbtxt')
NUM_CLASSES = 7

In [5]:
detection_graph = tf.Graph()
with detection_graph.as_default():
    od_graph_def = tf.GraphDef()
    with tf.gfile.GFile(PATH_TO_CKPT, 'rb') as fid:
        serialized_graph = fid.read()
        od_graph_def.ParseFromString(serialized_graph)
        tf.import_graph_def(od_graph_def, name='')

In [6]:
label_map = label_map_util.load_labelmap(PATH_TO_LABELS)
categories = label_map_util.convert_label_map_to_categories(label_map, max_num_classes=NUM_CLASSES, use_display_name=True)
category_index = label_map_util.create_category_index(categories)

In [7]:
def load_image_into_numpy_array(image):
    (im_width, im_height) = image.size
    return np.array(image.getdata()).reshape(
        (im_height, im_width, 3)).astype(np.uint8)

In [8]:
PATH_TO_TEST_IMAGES_DIR = 'test_images'
category = 'fishing'
TEST_IMAGE_PATHS = [ os.path.join(PATH_TO_TEST_IMAGES_DIR, '{}_{}.jpg'.format(category, i)) for i in range(1, 7) ]
IMAGE_SIZE = (12, 8)

with detection_graph.as_default():
    with tf.Session(graph=detection_graph) as sess:
        # Definite input and output Tensors for detection_graph
        image_tensor = detection_graph.get_tensor_by_name('image_tensor:0')
        # Each box represents a part of the image where a particular object was detected.
        detection_boxes = detection_graph.get_tensor_by_name('detection_boxes:0')
        # Each score represent how level of confidence for each of the objects.
        # Score is shown on the result image, together with the class label.
        detection_scores = detection_graph.get_tensor_by_name('detection_scores:0')
        detection_classes = detection_graph.get_tensor_by_name('detection_classes:0')
        num_detections = detection_graph.get_tensor_by_name('num_detections:0')
        for image_path in TEST_IMAGE_PATHS:
            image = Image.open(image_path)
            image_np = load_image_into_numpy_array(image)
            image_np_expanded = np.expand_dims(image_np, axis=0)
            (boxes, scores, classes, num) = sess.run(
                [detection_boxes, detection_scores, detection_classes, num_detections],
                feed_dict={image_tensor: image_np_expanded})
            vis_util.visualize_boxes_and_labels_on_image_array(
                image_np,
                np.squeeze(boxes),
                np.squeeze(classes).astype(np.int32),
                np.squeeze(scores),
                category_index,
                use_normalized_coordinates=True,
                min_score_thresh=.5,
                line_thickness=5)

            ################### Data analysis ###################
            get_scores = np.squeeze(scores)
            get_category = np.array([category_index.get(i) for i in classes[0]])
            get_boxes = np.squeeze(boxes)

            count_objects = 0
            count_category = np.array([])
            count_score = np.array([])
            for i in range(len(get_scores)):
                if scores is None or get_scores[i] > 0.5:
                    count_objects = count_objects + 1
                    count_category = np.append(count_category, get_category[i])
                    count_score = np.append(count_score, get_scores[i])
                    print(get_category[i])
            #####################################################
            
            try:
                score = (str(count_score[0]).split('.'))[1][:4]
            except Exception as e:
                score = 0
                
            plt.figure(figsize=IMAGE_SIZE)
            plt.imshow(image_np)
            result_path = 'test_images/result/'
            if not os.path.isdir(result_path):
                os.mkdir(result_path)            
            plt.savefig('{}{}_{}.jpg'.format(result_path, (image_path.split('\\')[-1]).split('.')[0], score), 
                        bbox_inches='tight',
                        dpi=300)

{'name': 'sailing', 'id': 4}
{'name': 'fishing', 'id': 1}
{'name': 'fishing', 'id': 1}
{'name': 'passenger', 'id': 3}
{'name': 'merchant', 'id': 2}


In [9]:
PATH_TO_TEST_IMAGES_DIR = 'test_images'
category = 'merchant'
TEST_IMAGE_PATHS = [ os.path.join(PATH_TO_TEST_IMAGES_DIR, '{}_{}.jpg'.format(category, i)) for i in range(1, 7) ]
IMAGE_SIZE = (12, 8)

with detection_graph.as_default():
    with tf.Session(graph=detection_graph) as sess:
        # Definite input and output Tensors for detection_graph
        image_tensor = detection_graph.get_tensor_by_name('image_tensor:0')
        # Each box represents a part of the image where a particular object was detected.
        detection_boxes = detection_graph.get_tensor_by_name('detection_boxes:0')
        # Each score represent how level of confidence for each of the objects.
        # Score is shown on the result image, together with the class label.
        detection_scores = detection_graph.get_tensor_by_name('detection_scores:0')
        detection_classes = detection_graph.get_tensor_by_name('detection_classes:0')
        num_detections = detection_graph.get_tensor_by_name('num_detections:0')
        for image_path in TEST_IMAGE_PATHS:
            image = Image.open(image_path)
            image_np = load_image_into_numpy_array(image)
            image_np_expanded = np.expand_dims(image_np, axis=0)
            (boxes, scores, classes, num) = sess.run(
                [detection_boxes, detection_scores, detection_classes, num_detections],
                feed_dict={image_tensor: image_np_expanded})
            vis_util.visualize_boxes_and_labels_on_image_array(
                image_np,
                np.squeeze(boxes),
                np.squeeze(classes).astype(np.int32),
                np.squeeze(scores),
                category_index,
                use_normalized_coordinates=True,
                min_score_thresh=.5,
                line_thickness=5)

            ################### Data analysis ###################
            get_scores = np.squeeze(scores)
            get_category = np.array([category_index.get(i) for i in classes[0]])
            get_boxes = np.squeeze(boxes)

            count_objects = 0
            count_category = np.array([])
            count_score = np.array([])
            for i in range(len(get_scores)):
                if scores is None or get_scores[i] > 0.5:
                    count_objects = count_objects + 1
                    count_category = np.append(count_category, get_category[i])
                    count_score = np.append(count_score, get_scores[i])
                    print(get_category[i])
            #####################################################
            
            try:
                score = (str(count_score[0]).split('.'))[1][:4]
            except Exception as e:
                score = 0
                
            plt.figure(figsize=IMAGE_SIZE)
            plt.imshow(image_np)
            result_path = 'test_images/result/'
            if not os.path.isdir(result_path):
                os.mkdir(result_path) 
            plt.savefig('{}{}_{}.jpg'.format(result_path, (image_path.split('\\')[-1]).split('.')[0], score), 
                        bbox_inches='tight',
                        dpi=300)

{'name': 'merchant', 'id': 2}
{'name': 'merchant', 'id': 2}
{'name': 'passenger', 'id': 3}
{'name': 'merchant', 'id': 2}
{'name': 'merchant', 'id': 2}
{'name': 'merchant', 'id': 2}
{'name': 'merchant', 'id': 2}


In [10]:
PATH_TO_TEST_IMAGES_DIR = 'test_images'
category = 'passenger'
TEST_IMAGE_PATHS = [ os.path.join(PATH_TO_TEST_IMAGES_DIR, '{}_{}.jpg'.format(category, i)) for i in range(1, 7) ]
IMAGE_SIZE = (12, 8)

with detection_graph.as_default():
    with tf.Session(graph=detection_graph) as sess:
        # Definite input and output Tensors for detection_graph
        image_tensor = detection_graph.get_tensor_by_name('image_tensor:0')
        # Each box represents a part of the image where a particular object was detected.
        detection_boxes = detection_graph.get_tensor_by_name('detection_boxes:0')
        # Each score represent how level of confidence for each of the objects.
        # Score is shown on the result image, together with the class label.
        detection_scores = detection_graph.get_tensor_by_name('detection_scores:0')
        detection_classes = detection_graph.get_tensor_by_name('detection_classes:0')
        num_detections = detection_graph.get_tensor_by_name('num_detections:0')
        for image_path in TEST_IMAGE_PATHS:
            image = Image.open(image_path)
            image_np = load_image_into_numpy_array(image)
            image_np_expanded = np.expand_dims(image_np, axis=0)
            (boxes, scores, classes, num) = sess.run(
                [detection_boxes, detection_scores, detection_classes, num_detections],
                feed_dict={image_tensor: image_np_expanded})
            vis_util.visualize_boxes_and_labels_on_image_array(
                image_np,
                np.squeeze(boxes),
                np.squeeze(classes).astype(np.int32),
                np.squeeze(scores),
                category_index,
                use_normalized_coordinates=True,
                min_score_thresh=.5,
                line_thickness=5)

            ################### Data analysis ###################
            get_scores = np.squeeze(scores)
            get_category = np.array([category_index.get(i) for i in classes[0]])
            get_boxes = np.squeeze(boxes)

            count_objects = 0
            count_category = np.array([])
            count_score = np.array([])
            for i in range(len(get_scores)):
                if scores is None or get_scores[i] > 0.5:
                    count_objects = count_objects + 1
                    count_category = np.append(count_category, get_category[i])
                    count_score = np.append(count_score, get_scores[i])
                    print(get_category[i])
            #####################################################
            
            try:
                score = (str(count_score[0]).split('.'))[1][:4]
            except Exception as e:
                score = 0
                
            plt.figure(figsize=IMAGE_SIZE)
            plt.imshow(image_np)
            result_path = 'test_images/result/'
            if not os.path.isdir(result_path):
                os.mkdir(result_path) 
            plt.savefig('{}{}_{}.jpg'.format(result_path, (image_path.split('\\')[-1]).split('.')[0], score), 
                        bbox_inches='tight',
                        dpi=300)

{'name': 'passenger', 'id': 3}
{'name': 'passenger', 'id': 3}
{'name': 'passenger', 'id': 3}
{'name': 'passenger', 'id': 3}
{'name': 'passenger', 'id': 3}
{'name': 'passenger', 'id': 3}


In [11]:
PATH_TO_TEST_IMAGES_DIR = 'test_images'
category = 'sailing'
TEST_IMAGE_PATHS = [ os.path.join(PATH_TO_TEST_IMAGES_DIR, '{}_{}.jpg'.format(category, i)) for i in range(1, 7) ]
IMAGE_SIZE = (12, 8)

with detection_graph.as_default():
    with tf.Session(graph=detection_graph) as sess:
        # Definite input and output Tensors for detection_graph
        image_tensor = detection_graph.get_tensor_by_name('image_tensor:0')
        # Each box represents a part of the image where a particular object was detected.
        detection_boxes = detection_graph.get_tensor_by_name('detection_boxes:0')
        # Each score represent how level of confidence for each of the objects.
        # Score is shown on the result image, together with the class label.
        detection_scores = detection_graph.get_tensor_by_name('detection_scores:0')
        detection_classes = detection_graph.get_tensor_by_name('detection_classes:0')
        num_detections = detection_graph.get_tensor_by_name('num_detections:0')
        for image_path in TEST_IMAGE_PATHS:
            image = Image.open(image_path)
            image_np = load_image_into_numpy_array(image)
            image_np_expanded = np.expand_dims(image_np, axis=0)
            (boxes, scores, classes, num) = sess.run(
                [detection_boxes, detection_scores, detection_classes, num_detections],
                feed_dict={image_tensor: image_np_expanded})
            vis_util.visualize_boxes_and_labels_on_image_array(
                image_np,
                np.squeeze(boxes),
                np.squeeze(classes).astype(np.int32),
                np.squeeze(scores),
                category_index,
                use_normalized_coordinates=True,
                min_score_thresh=.5,
                line_thickness=5)

            ################### Data analysis ###################
            get_scores = np.squeeze(scores)
            get_category = np.array([category_index.get(i) for i in classes[0]])
            get_boxes = np.squeeze(boxes)

            count_objects = 0
            count_category = np.array([])
            count_score = np.array([])
            for i in range(len(get_scores)):
                if scores is None or get_scores[i] > 0.5:
                    count_objects = count_objects + 1
                    count_category = np.append(count_category, get_category[i])
                    count_score = np.append(count_score, get_scores[i])
                    print(get_category[i])
            #####################################################
            
            try:
                score = (str(count_score[0]).split('.'))[1][:4]
            except Exception as e:
                score = 0
                
            plt.figure(figsize=IMAGE_SIZE)
            plt.imshow(image_np)
            result_path = 'test_images/result/'
            if not os.path.isdir(result_path):
                os.mkdir(result_path) 
            plt.savefig('{}{}_{}.jpg'.format(result_path, (image_path.split('\\')[-1]).split('.')[0], score), 
                        bbox_inches='tight',
                        dpi=300)

{'name': 'sailing', 'id': 4}
{'name': 'sailing', 'id': 4}
{'name': 'sailing', 'id': 4}


c:\users\neark\realtimeshipdetector\env\lib\site-packages\matplotlib\pyplot.py:514: RuntimeWarning: More than 20 figures have been opened. Figures created through the pyplot interface (`matplotlib.pyplot.figure`) are retained until explicitly closed and may consume too much memory. (To control this warning, see the rcParam `figure.max_open_warning`).
  max_open_warning, RuntimeWarning)


{'name': 'sailing', 'id': 4}
{'name': 'sailing', 'id': 4}
{'name': 'sailing', 'id': 4}


In [12]:
PATH_TO_TEST_IMAGES_DIR = 'test_images'
category = 'small'
TEST_IMAGE_PATHS = [ os.path.join(PATH_TO_TEST_IMAGES_DIR, '{}_{}.jpg'.format(category, i)) for i in range(1, 7) ]
IMAGE_SIZE = (12, 8)

with detection_graph.as_default():
    with tf.Session(graph=detection_graph) as sess:
        # Definite input and output Tensors for detection_graph
        image_tensor = detection_graph.get_tensor_by_name('image_tensor:0')
        # Each box represents a part of the image where a particular object was detected.
        detection_boxes = detection_graph.get_tensor_by_name('detection_boxes:0')
        # Each score represent how level of confidence for each of the objects.
        # Score is shown on the result image, together with the class label.
        detection_scores = detection_graph.get_tensor_by_name('detection_scores:0')
        detection_classes = detection_graph.get_tensor_by_name('detection_classes:0')
        num_detections = detection_graph.get_tensor_by_name('num_detections:0')
        for image_path in TEST_IMAGE_PATHS:
            image = Image.open(image_path)
            image_np = load_image_into_numpy_array(image)
            image_np_expanded = np.expand_dims(image_np, axis=0)
            (boxes, scores, classes, num) = sess.run(
                [detection_boxes, detection_scores, detection_classes, num_detections],
                feed_dict={image_tensor: image_np_expanded})
            vis_util.visualize_boxes_and_labels_on_image_array(
                image_np,
                np.squeeze(boxes),
                np.squeeze(classes).astype(np.int32),
                np.squeeze(scores),
                category_index,
                use_normalized_coordinates=True,
                min_score_thresh=.5,
                line_thickness=5)

            ################### Data analysis ###################
            get_scores = np.squeeze(scores)
            get_category = np.array([category_index.get(i) for i in classes[0]])
            get_boxes = np.squeeze(boxes)

            count_objects = 0
            count_category = np.array([])
            count_score = np.array([])
            for i in range(len(get_scores)):
                if scores is None or get_scores[i] > 0.5:
                    count_objects = count_objects + 1
                    count_category = np.append(count_category, get_category[i])
                    count_score = np.append(count_score, get_scores[i])
                    print(get_category[i])
            #####################################################
            
            try:
                score = (str(count_score[0]).split('.'))[1][:4]
            except Exception as e:
                score = 0
                
            plt.figure(figsize=IMAGE_SIZE)
            plt.imshow(image_np)
            result_path = 'test_images/result/'
            if not os.path.isdir(result_path):
                os.mkdir(result_path) 
            plt.savefig('{}{}_{}.jpg'.format(result_path, (image_path.split('\\')[-1]).split('.')[0], score), 
                        bbox_inches='tight',
                        dpi=300)

{'name': 'small', 'id': 5}
{'name': 'passenger', 'id': 3}
{'name': 'small', 'id': 5}
{'name': 'small', 'id': 5}
{'name': 'small', 'id': 5}


In [13]:
PATH_TO_TEST_IMAGES_DIR = 'test_images'
category = 'tug'
TEST_IMAGE_PATHS = [ os.path.join(PATH_TO_TEST_IMAGES_DIR, '{}_{}.jpg'.format(category, i)) for i in range(1, 7) ]
IMAGE_SIZE = (12, 8)

with detection_graph.as_default():
    with tf.Session(graph=detection_graph) as sess:
        # Definite input and output Tensors for detection_graph
        image_tensor = detection_graph.get_tensor_by_name('image_tensor:0')
        # Each box represents a part of the image where a particular object was detected.
        detection_boxes = detection_graph.get_tensor_by_name('detection_boxes:0')
        # Each score represent how level of confidence for each of the objects.
        # Score is shown on the result image, together with the class label.
        detection_scores = detection_graph.get_tensor_by_name('detection_scores:0')
        detection_classes = detection_graph.get_tensor_by_name('detection_classes:0')
        num_detections = detection_graph.get_tensor_by_name('num_detections:0')
        for image_path in TEST_IMAGE_PATHS:
            image = Image.open(image_path)
            image_np = load_image_into_numpy_array(image)
            image_np_expanded = np.expand_dims(image_np, axis=0)
            (boxes, scores, classes, num) = sess.run(
                [detection_boxes, detection_scores, detection_classes, num_detections],
                feed_dict={image_tensor: image_np_expanded})
            vis_util.visualize_boxes_and_labels_on_image_array(
                image_np,
                np.squeeze(boxes),
                np.squeeze(classes).astype(np.int32),
                np.squeeze(scores),
                category_index,
                use_normalized_coordinates=True,
                min_score_thresh=.5,
                line_thickness=5)

            ################### Data analysis ###################
            get_scores = np.squeeze(scores)
            get_category = np.array([category_index.get(i) for i in classes[0]])
            get_boxes = np.squeeze(boxes)

            count_objects = 0
            count_category = np.array([])
            count_score = np.array([])
            for i in range(len(get_scores)):
                if scores is None or get_scores[i] > 0.5:
                    count_objects = count_objects + 1
                    count_category = np.append(count_category, get_category[i])
                    count_score = np.append(count_score, get_scores[i])
                    print(get_category[i])
            #####################################################
            
            try:
                score = (str(count_score[0]).split('.'))[1][:4]
            except Exception as e:
                score = 0
                
            plt.figure(figsize=IMAGE_SIZE)
            plt.imshow(image_np)
            result_path = 'test_images/result/'
            if not os.path.isdir(result_path):
                os.mkdir(result_path) 
            plt.savefig('{}{}_{}.jpg'.format(result_path, (image_path.split('\\')[-1]).split('.')[0], score), 
                        bbox_inches='tight',
                        dpi=300)

{'name': 'tug', 'id': 6}
{'name': 'tug', 'id': 6}
{'name': 'tug', 'id': 6}
{'name': 'tug', 'id': 6}
{'name': 'tug', 'id': 6}
{'name': 'tug', 'id': 6}


In [14]:
PATH_TO_TEST_IMAGES_DIR = 'test_images'
category = 'warship'
TEST_IMAGE_PATHS = [ os.path.join(PATH_TO_TEST_IMAGES_DIR, '{}_{}.jpg'.format(category, i)) for i in range(1, 7) ]
IMAGE_SIZE = (12, 8)

with detection_graph.as_default():
    with tf.Session(graph=detection_graph) as sess:
        # Definite input and output Tensors for detection_graph
        image_tensor = detection_graph.get_tensor_by_name('image_tensor:0')
        # Each box represents a part of the image where a particular object was detected.
        detection_boxes = detection_graph.get_tensor_by_name('detection_boxes:0')
        # Each score represent how level of confidence for each of the objects.
        # Score is shown on the result image, together with the class label.
        detection_scores = detection_graph.get_tensor_by_name('detection_scores:0')
        detection_classes = detection_graph.get_tensor_by_name('detection_classes:0')
        num_detections = detection_graph.get_tensor_by_name('num_detections:0')
        for image_path in TEST_IMAGE_PATHS:
            image = Image.open(image_path)
            image_np = load_image_into_numpy_array(image)
            image_np_expanded = np.expand_dims(image_np, axis=0)
            (boxes, scores, classes, num) = sess.run(
                [detection_boxes, detection_scores, detection_classes, num_detections],
                feed_dict={image_tensor: image_np_expanded})
            vis_util.visualize_boxes_and_labels_on_image_array(
                image_np,
                np.squeeze(boxes),
                np.squeeze(classes).astype(np.int32),
                np.squeeze(scores),
                category_index,
                use_normalized_coordinates=True,
                min_score_thresh=.5,
                line_thickness=5)

            ################### Data analysis ###################
            get_scores = np.squeeze(scores)
            get_category = np.array([category_index.get(i) for i in classes[0]])
            get_boxes = np.squeeze(boxes)

            count_objects = 0
            count_category = np.array([])
            count_score = np.array([])
            for i in range(len(get_scores)):
                if scores is None or get_scores[i] > 0.5:
                    count_objects = count_objects + 1
                    count_category = np.append(count_category, get_category[i])
                    count_score = np.append(count_score, get_scores[i])
                    print(get_category[i])
            #####################################################
            
            try:
                score = (str(count_score[0]).split('.'))[1][:4]
            except Exception as e:
                score = 0
                
            plt.figure(figsize=IMAGE_SIZE)
            plt.imshow(image_np)
            result_path = 'test_images/result/'
            if not os.path.isdir(result_path):
                os.mkdir(result_path) 
            plt.savefig('{}{}_{}.jpg'.format(result_path, (image_path.split('\\')[-1]).split('.')[0], score), 
                        bbox_inches='tight',
                        dpi=300)

{'name': 'warship', 'id': 7}
{'name': 'warship', 'id': 7}
{'name': 'warship', 'id': 7}
{'name': 'warship', 'id': 7}
{'name': 'warship', 'id': 7}
